In [2]:
import gymnasium as gym
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, Dense
from keras.optimizers import Adam
from gymnasium.wrappers import FrameStack
from tqdm import tqdm

In [3]:
env = gym.make("ALE/Tetris-v5", render_mode='human')
env.reset()
done = False

In [4]:
state_size = env.observation_space.shape
action_size = env.action_space.n
batch_size = 64
gamma = 0.95
episodes = 10
learning_rate = 0.001

In [5]:
def build_model(input_shape, action_size):
    model = Sequential([ # Each person should change the amount of Conv2D/Dense layers, as well as the filter amount and kernel_size/strides
        Conv2D(32, kernel_size=(8, 8), strides=(4, 4), activation='relu', input_shape=input_shape),
        Conv2D(64, kernel_size=(4, 4), strides=(2, 2), activation='relu'),
        Conv2D(64, kernel_size=(3, 3), strides=(1, 1), activation='relu'),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dense(action_size, activation='linear')
    ])
    # model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model

In [6]:
dqn = build_model(state_size, action_size)
dqn.load_weights('brennan.weights.h5')

c:\Users\xygni\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [7]:
def preprocess_rgb(state):
    state = cv2.resize(state, (84, 110))
    state = cv2.cvtColor(state, cv2.COLOR_BGR2GRAY)
    state = state[26:110, :]
    state = state.astype(np.float32) / 255.0
    return state.reshape(84, 84, 1)

In [8]:
def preprocess_grayscale(state):
    if state is None:
        return np.zeros((84, 84, 1), dtype=np.float32)  # Return a blank image if state is None
    state = cv2.cvtColor(state, cv2.COLOR_RGB2GRAY) if state.shape[-1] == 3 else state  # Convert RGB to grayscale if needed
    state = cv2.resize(state, (84, 84))  # Resize the image
    state = state.astype(np.float32) / 255.0  # Normalize pixel values
    return np.expand_dims(state, axis=-1)  # Ensure the image has a single channel

In [9]:
# Take a random action
action = env.action_space.sample()

# Perform the action and capture the return values in a single variable
info = env.step(action)

(array([[[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [ 50, 132,  50],
         [ 50, 132,  50],
         [ 50, 132,  50]],
 
        ...,
 
        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0]],
 
        [[  0,   0,   0],
         [  0,   0,   0],
         [  0,   0,   0],
         ...,
         [  0,   0,   0],
  

In [10]:
for episode in tqdm(range(episodes), desc='Episode Progress'):
    state, _ = env.reset()
    done = False
    while not done:
        q_values = dqn.predict(np.array([state]), verbose=None)[0]
        action = np.argmax(q_values)  # Exploitation

        # Ensure action is within bounds
        action = np.clip(action, 0, action_size - 1)

        next_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated

        state = next_state

        if done:
            break
env.close()

Episode Progress:   4%|▍         | 4/100 [10:56<4:27:43, 167.32s/it]